# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sun Sep 19 20:24:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.49       Driver Version: 457.49       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   43C    P8     2W /  N/A |    366MiB /  4096MiB |     39%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2019년 Celltrion Data**

In [5]:
lexicon_2019 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2019_celltrion.csv')

lexicon_2019.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,셀트리온,68270,매일경제,2019010104,글로벌 불확실성·기업실적 둔화…`상저하고` 장세 전망 우세,http://news.mk.co.kr/newsRead.php?year=2019&no...,◆ 2019 경제기상도 / 증시 ◆ \r\n\n\n 주요 증권사 리서치센터에 따르면...,2019-01-02,4,210570,211976,199783,201192,1045357,-0.035952,-1,0,경제 기상도 증시 주요 증권사 리서치 센터 국내 증시 코스피 미만 저하 기록 전망 ...


In [6]:
lexicon_2019[lexicon_2019['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2019.dropna(axis=0, inplace=True)
lexicon_2019 = lexicon_2019.reset_index(drop=True)
lexicon_2019.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
neg_lexicon_2019 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_neg_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_neg와의 Cosine Similarity 계산
for x in range(len(lexicon_2019['Tokenization'])):
    news_num = x+1
    for y in range(len(negative)):
        for z in range(len(list(set(lexicon_2019['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(negative[y], list(set(lexicon_2019['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(negative[y], list(set(lexicon_2019['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2019['Tokenization'][x].split())):
                    if lexicon_2019['Tokenization'][x].split()[w] == list(set(lexicon_2019['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2019['date'][x],
                    'news_num': news_num,
                    'KOSELF_neg_word': negative[y],
                    'news_word': list(set(lexicon_2019['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(negative[y], list(set(lexicon_2019['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                neg_lexicon_2019 = neg_lexicon_2019.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2019['date'][x], negative[y], list(set(lexicon_2019['Tokenization'][x].split()))[z], ko_model.wv.similarity(negative[y], list(set(lexicon_2019['Tokenization'][x].split()))[z])))

***2019-01-02 Cosine Similarity between <둔화> & <악화> : 0.5065981149673462
***2019-01-02 Cosine Similarity between <둔화> & <업황> : 0.5578197240829468
***2019-01-02 Cosine Similarity between <둔화> & <반등> : 0.5092458724975586
***2019-01-02 Cosine Similarity between <둔화> & <급락> : 0.5491443276405334
***2019-01-02 Cosine Similarity between <악화> & <둔화> : 0.5065981149673462
***2019-01-02 Cosine Similarity between <약화> & <저하> : 0.5494820475578308
***2019-01-02 Cosine Similarity between <약화> & <악화> : 0.5064777731895447
***2019-01-02 Cosine Similarity between <의혹> & <논란> : 0.5190056562423706
***2019-01-02 Cosine Similarity between <침체> & <경기> : 0.5360626578330994
***2019-01-02 Cosine Similarity between <침체> & <불황> : 0.6393564939498901
***2019-01-02 Cosine Similarity between <침체> & <둔화> : 0.5701860189437866
***2019-01-02 Cosine Similarity between <침체> & <업황> : 0.5205410122871399
***2019-01-02 Cosine Similarity between <하락> & <저하> : 0.5286633372306824
***2019-01-02 Cosine Similarity between <하락> & <상승>

In [11]:
neg_lexicon_2019

,date,news_num,KOSELF_neg_word,news_word,cosine_similarity,frequency
0,2019-01-02,1,둔화,악화,0.506598,1
1,2019-01-02,1,둔화,업황,0.557820,1
2,2019-01-02,1,둔화,반등,0.509246,3
3,2019-01-02,1,둔화,급락,0.549144,1
4,2019-01-02,1,악화,둔화,0.506598,2
...,...,...,...,...,...,...
11112,2020-01-02,1698,둔화,증가,0.504186,1
11113,2020-01-02,1698,하락,상승,0.712648,4
11114,2020-01-02,1698,하락,증가,0.553474,1
11115,2020-01-02,1699,실패,성공,0.706813,1


In [12]:
neg_lexicon_2019.to_csv('../../../../Code/Data/Test/Stock-Year/neg_celltrion_2019.csv', index=False)

In [13]:
a = list(set(list(neg_lexicon_2019['news_word'])))

a

['등락',
 '위기',
 '금값',
 '급감',
 '호전',
 '불황',
 '경기',
 '운항',
 '둔화',
 '눈총',
 '부도',
 '보상',
 '플러스',
 '폐업',
 '급등',
 '혼돈',
 '강점',
 '저하',
 '악재',
 '절상',
 '격벽',
 '유발',
 '약화',
 '선박',
 '지연',
 '급락',
 '조아연',
 '증가',
 '악화',
 '조처',
 '차량',
 '논란',
 '업황',
 '정체',
 '의혹',
 '상승',
 '견조',
 '의심',
 '호황',
 '누전',
 '참패',
 '감소',
 '손해배상',
 '약세',
 '강화',
 '류현지',
 '연루',
 '폭락',
 '위축',
 '불안',
 '역풍',
 '보합',
 '성공',
 '폭로',
 '비리',
 '외압',
 '침체',
 '하락',
 '주춤',
 '질책',
 '반등',
 '폭등',
 '끝내',
 '좌절',
 '화물',
 '조치']

In [14]:
b = list(set(list(neg_lexicon_2019[neg_lexicon_2019['cosine_similarity']>=0.7]['news_word'])))

b

['침체', '하락', '약화', '상승', '의심', '지연', '성공', '정체', '의혹', '조치']

In [15]:
c = list(set(list(neg_lexicon_2019[neg_lexicon_2019['cosine_similarity']>=0.65]['news_word'])))

c

['조치', '침체', '하락', '약화', '의심', '지연', '급락', '조처', '성공', '정체', '의혹', '상승']